# TW test

## Resampling Techniques for class impalanced & Support vector machine (SVM)

###  Split the Data into Training and Testing

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
binary_df = pd.read_csv('../resources/binary_classifier.csv')
print(binary_df.shape) # columns & row 
binary_df.head()

(122956, 18)


,pub_agency_name,agency_type_name,state_name,division_name,county_name,region_name,population_group_code,offense_code,offender_race,offender_ethnicity,offender_age,offender_sex,victim_type_code,location_code,weapon_code,prop_desc_code,stolen_value,recovered_flag
0,Angelina,County,Texas,West South Central,ANGELINA,South,8B,26B,Unknown,Unknown,0.0,U,I,25,95,20,375.0,False
1,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23H,Unknown,Unknown,0.0,U,I,20,95,77,1.0,False
2,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23H,Unknown,Unknown,0.0,U,I,20,95,65,0.0,False
3,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23H,Unknown,Unknown,0.0,U,I,20,95,13,320.0,False
4,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23F,Unknown,Unknown,0.0,U,I,20,95,77,1.0,False


In [4]:
# Transform text/object into numerical data for features to be used. 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
binaryEN_df = binary_df.copy()

binaryEN_df['population_group_code'] = le.fit_transform(binary_df['population_group_code'])
binaryEN_df['offense_code'] = le.fit_transform(binary_df['offense_code'])
binaryEN_df['offender_race'] = le.fit_transform(binary_df['offender_race'])
binaryEN_df['offender_ethnicity'] = le.fit_transform(binary_df['offender_ethnicity'])
binaryEN_df['offender_sex'] = le.fit_transform(binary_df['offender_sex'])
binaryEN_df['victim_type_code'] = le.fit_transform(binary_df['victim_type_code'])
binaryEN_df['recovered_flag'] = le.fit_transform(binary_df['recovered_flag'])

binaryEN_df.head()

,pub_agency_name,agency_type_name,state_name,division_name,county_name,region_name,population_group_code,offense_code,offender_race,offender_ethnicity,offender_age,offender_sex,victim_type_code,location_code,weapon_code,prop_desc_code,stolen_value,recovered_flag
0,Angelina,County,Texas,West South Central,ANGELINA,South,11,13,4,3,0.0,2,3,25,95,20,375.0,0
1,Jefferson,County,Alabama,East South Central,JEFFERSON,South,15,9,4,3,0.0,2,3,20,95,77,1.0,0
2,Jefferson,County,Alabama,East South Central,JEFFERSON,South,15,9,4,3,0.0,2,3,20,95,65,0.0,0
3,Jefferson,County,Alabama,East South Central,JEFFERSON,South,15,9,4,3,0.0,2,3,20,95,13,320.0,0
4,Jefferson,County,Alabama,East South Central,JEFFERSON,South,15,7,4,3,0.0,2,3,20,95,77,1.0,0


In [5]:
binaryEN_df.dtypes

pub_agency_name           object
agency_type_name          object
state_name                object
division_name             object
county_name               object
region_name               object
population_group_code      int32
offense_code               int32
offender_race              int32
offender_ethnicity         int32
offender_age             float64
offender_sex               int32
victim_type_code           int32
location_code              int64
weapon_code               object
prop_desc_code             int64
stolen_value             float64
recovered_flag             int64
dtype: object

In [7]:
# Create our features
# YOUR CODE HERE
X = binaryEN_df[['offense_code','offender_age','location_code','prop_desc_code','population_group_code', 'victim_type_code']] 
    
# Create our target 
y = binaryEN_df['recovered_flag']


In [8]:
X.describe()

,offense_code,offender_age,location_code,prop_desc_code,population_group_code,victim_type_code
count,122956.000000,122956.00000,122956.000000,122956.000000,122956.000000,122956.000000
mean,6.938604,12.60581,18.745014,34.192231,7.541405,2.405535
std,3.808066,18.07965,8.814946,27.089887,4.783051,1.404634
min,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000
25%,5.000000,0.00000,14.000000,13.000000,4.000000,3.000000
50%,7.000000,0.00000,20.000000,24.000000,6.000000,3.000000
75%,9.000000,26.00000,20.000000,65.000000,11.000000,3.000000
max,18.000000,99.00000,58.000000,99.000000,19.000000,8.000000


In [9]:
# Check the balance of our target values - 0 = Unrecovered, 2 = recovered
y.value_counts()

0    105675
1     17281
Name: recovered_flag, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({0: 79288, 1: 12929})

###  SMOTE Oversampling

In [11]:
# Resample the training data with SMOTE/synthetic minority oversampling technique
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,sampling_strategy='auto').fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 79288, 1: 79288})

In [12]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [13]:
# Display the confusion matrix
# YOUR CODE HERE
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,18670,7717
Actual 1,1889,2463


In [14]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
acc_score = balanced_accuracy_score(y_test, y_pred)

In [15]:
# Print the imbalanced classification report
# YOUR CODE HERE
from imblearn.metrics import classification_report_imbalanced
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,18670,7717
Actual 1,1889,2463


Accuracy Score : 0.6367460366861244
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.71      0.57      0.80      0.63      0.41     26387
          1       0.24      0.57      0.71      0.34      0.63      0.39      4352

avg / total       0.81      0.69      0.59      0.73      0.63      0.40     30739



### Undersampling --> PC crashed

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete

## Running too long 

#from imblearn.under_sampling import ClusterCentroids
#cc = ClusterCentroids(random_state=1)
#X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

#Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
ucm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
ucm_df = pd.DataFrame(ucm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

ucm_df

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
uacc_score = balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Confusion Matrix")
display(ucm_df)
print(f"Accuracy Score : {uacc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

### Combination (Over and Under) Sampling - SMOTEENN

from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y) 

Counter(y_resampled)

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
stcm = confusion_matrix(y_test, y_pred)

# Create a DataFrame from the confusion matrix.
stcm_df = pd.DataFrame(stcm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

stcm_df

,Predicted 0,Predicted 1
Actual 0,18620,7767
Actual 1,1900,2452


In [19]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
stacc_score = balanced_accuracy_score(y_test, y_pred)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print("Confusion Matrix")
display(stcm_df)
print(f"Accuracy Score : {stacc_score}")
print("Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,18620,7767
Actual 1,1900,2452


Accuracy Score : 0.6345348136838773
Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.91      0.71      0.56      0.79      0.63      0.40     26387
          1       0.24      0.56      0.71      0.34      0.63      0.39      4352

avg / total       0.81      0.69      0.58      0.73      0.63      0.40     30739



## Support vector machine (SVM)

In [23]:
from pathlib import Path
import numpy as np
import pandas as pd

data = Path('../resources/binary_classifier.csv')
binary_df = pd.read_csv(data)
print(binary_df.shape) # columns & row 
binary_df.head()

(122956, 18)


,pub_agency_name,agency_type_name,state_name,division_name,county_name,region_name,population_group_code,offense_code,offender_race,offender_ethnicity,offender_age,offender_sex,victim_type_code,location_code,weapon_code,prop_desc_code,stolen_value,recovered_flag
0,Angelina,County,Texas,West South Central,ANGELINA,South,8B,26B,Unknown,Unknown,0.0,U,I,25,95,20,375.0,False
1,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23H,Unknown,Unknown,0.0,U,I,20,95,77,1.0,False
2,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23H,Unknown,Unknown,0.0,U,I,20,95,65,0.0,False
3,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23H,Unknown,Unknown,0.0,U,I,20,95,13,320.0,False
4,Jefferson,County,Alabama,East South Central,JEFFERSON,South,9A,23F,Unknown,Unknown,0.0,U,I,20,95,77,1.0,False


In [24]:
y = binary_df["recovered_flag"]
X = binary_df[['offense_code','offender_age','location_code','prop_desc_code','population_group_code', 'victim_type_code']]     

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

(92217, 6)

In [26]:
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [27]:
model.fit(X_train, y_train)

ValueError: could not convert string to float: '23F'

In [28]:
y_pred = model.predict(X_test)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test
}).reset_index(drop=True)
results.head()

ValueError: could not convert string to float: '26F'

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)